In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
path='/kaggle/input/finance-accounting-courses-udemy-13k-course/udemy_output_All_Finance__Accounting_p1_p626.csv'
udemy = pd.read_csv(path)

In [ ]:
udemy.head(5)

In [ ]:
udemy.info()

In [ ]:
udemy.duplicated().sum()

There are 19 variables i nthis dataset and without missing values or duplicated records. 'created' and 'published_time' should be changed into datetime.

In [ ]:
udemy['created']= pd.to_datetime(udemy['created'])
udemy['published_time']= pd.to_datetime(udemy['published_time'])
udemy['published_year']= udemy['published_time'].dt.year
udemy['published_month']= udemy['published_time'].dt.month

 Now I want to know about difference about courses accordign to column 'is_paid'.

In [ ]:
udemy_pay = udemy.groupby(['published_year','is_paid']).agg({'rating':'mean','is_wishlisted':'count','num_subscribers':'mean',
                                                              'num_published_lectures':'mean','num_published_practice_tests':'mean'}).reset_index()

In [ ]:
udemy_pay.style.background_gradient(cmap='Greens',axis =0,low=0,high=1)

There is a trend for paid courses to test subscribers. 

In [ ]:
import matplotlib.pyplot as plt
udemy_pay.pivot('published_year','is_paid','rating').plot(kind='bar')
plt.legend()
plt.ylim(0,7.0)
plt.ylabel('average rating')
plt.show()


average rating plot shows no large difference between paid and un_paid courses. Even un_paid courses have better rating recently and in general. 

In [ ]:
udemy_pay.pivot('published_year','is_paid','num_subscribers').plot(kind='bar')
plt.legend()

plt.ylabel('num_subscribers')
plt.show()

less and less subscribers for courses. Now more specific growth or decline rate is needed.

In [ ]:
from matplotlib import colors

def background_gradient(s, m, M, cmap='PuBu', low=0, high=0):
    rng = M - m
    norm = colors.Normalize(m - (rng * low),
                            M + (rng * high))
    normed = norm(s.values)
    c = [colors.rgb2hex(x) for x in plt.cm.get_cmap(cmap)(normed)]
    return ['background-color: %s' % color for color in c]

df=pd.pivot_table(data=udemy_pay,index='is_paid',columns='published_year',values='num_subscribers').pct_change(axis=1)
df

df.style.apply(background_gradient,
               cmap='PuBu',
               m=df.min().min(),
               M=df.max().max(),
               low=0,
               high=0.5)

It is clear that in 2013 are highest growth rates for both of them. 2020 is highst decline rate for un_paid courses and 2012 for paid courses. A good news is that decline rate for paid courses becomes smaller recently.

In [ ]:
udemy_pay.pivot('published_year','is_paid','is_wishlisted').plot(kind='bar')
plt.legend()

plt.ylabel('is_wishlisted')
plt.show()

more and more need-paying courses are wishlisted, which shows a potential oppotunity for a promotion to improve sales performance.

In [ ]:
udemy_pay.pivot('published_year','is_paid','num_published_lectures').plot(kind='bar')
plt.legend()

plt.ylabel('num_published_lectures')
plt.show()

In [ ]:
Almost two times more lectures from paid courses than un_paid courses are published. 

Conclusion: more and more published courses are challeging the attraction for each course. The reaction to this challenge are preparing more lectures and tests for subscribers, which successfully absorbs learners from un_paid courses. For more and larger improvement, I suggest a discount for those courses in the wishlist.  

Now starts a sales analysis for paid courses.  

In [ ]:
sales_list=['published_year','title','avg_rating','avg_rating_recent','num_subscribers','num_reviews','discount_price__amount'
           ,'discount_price__currency','price_detail__amount']
sales = udemy[udemy['is_paid']==True][sales_list]

In [ ]:
sales.head(5)

In [ ]:
sales['rating_growth']= sales['avg_rating_recent']/sales['avg_rating']-1
sales_better_rating=sales[sales['rating_growth']>=0]
sales_worse_rating=sales[sales['rating_growth']<0]

In [ ]:
sales_better_rating.head(5)

In [ ]:
sales_worse_rating.head(5)

top 10 course in term of subscribers

In [ ]:
sales_better_rating.sort_values('num_subscribers',ascending=False)['title'].head(10)

In [ ]:
subscriber_list=sales_better_rating.sort_values('num_subscribers',ascending=False)['title'].head(10).reset_index(drop=True)

rating_list=sales_better_rating.sort_values('rating_growth',ascending=False)['title'].head(10).reset_index(drop=True)

In [ ]:
combine=pd.DataFrame(
{
    'rating_list':rating_list,
    'subscriber_list':subscriber_list
})
combine

In [ ]:
subscriber_list2=sales_worse_rating.sort_values('num_subscribers',ascending=False)['title'].head(10).reset_index(drop=True)

rating_list2=sales_worse_rating.sort_values('rating_growth',ascending=False)['title'].head(10).reset_index(drop=True)
combine2=pd.DataFrame(
{
    'rating_list':rating_list2,
    'subscriber_list':subscriber_list2
})
combine2

Compare these two lists, it tells us that most people go to courses about data and finance industry. Data becomes a trend recently. This phenomenon shows in negative growth rate group as well.However, for course quality,there are courses with high quanlity in other branches like design, blogger. Now i wonder the difference between top 10 and last 10 courses in term of subscriber and rating growth.

In [ ]:
subscriber_list3=sales_better_rating.sort_values('num_subscribers',ascending=True)['title'].head(10).reset_index(drop=True)

rating_list3=sales_better_rating.sort_values('rating_growth',ascending=True)['title'].head(10).reset_index(drop=True)
subscriber_list4=sales_worse_rating.sort_values('num_subscribers',ascending=True)['title'].head(10).reset_index(drop=True)

rating_list4=sales_worse_rating.sort_values('rating_growth',ascending=True)['title'].head(10).reset_index(drop=True)


In [ ]:
top_10= rating_list.append([rating_list2,subscriber_list,subscriber_list2]).reset_index(drop=True)
last_10 =subscriber_list3.append([subscriber_list4, rating_list3 , rating_list4]).reset_index(drop=True)

In [ ]:
top_10.duplicated().sum()
last_10.duplicated().sum()

In [ ]:
top_10_course=udemy.merge(top_10,on='title',how='inner')
last_10_course=udemy.merge(last_10,on='title',how='inner')

In [ ]:
top_10_course[['num_published_lectures','num_published_practice_tests' ,
     'discount_price__amount','price_detail__amount']].mean()

In [ ]:
last_10_course[['num_published_lectures','num_published_practice_tests' ,
     'discount_price__amount','price_detail__amount']].mean()

despite of higher price and less discount, those top 10 courses win subscribers and reputation with high quanlity like more lectures and tests. Is is really important factors for creating popular courses.